In [1]:
import numpy as np
import os
import pulp
import pandas as pd
import openpyxl

from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.pyplot import MultipleLocator

In [2]:
# process price data, select prices with label 1

hours = list(map(str, range(24))) 
prices_columns = list(map(str, range(24))) + ['label']
prices = pd.read_csv('./TestingResults.txt', names = prices_columns)
prices = prices.loc[prices['label']==1]
prices.pop('label')


0     1
5     1
8     1
12    1
15    1
16    1
17    1
18    1
19    1
21    1
27    1
32    1
34    1
38    1
47    1
48    1
50    1
54    1
55    1
56    1
63    1
64    1
67    1
68    1
69    1
73    1
76    1
77    1
79    1
83    1
85    1
89    1
92    1
93    1
94    1
95    1
Name: label, dtype: int64

In [3]:
# process user data, collect tasks

usertask = pd.read_excel("./COMP3217CW2Input.xlsx", sheet_name = 'User & Task ID')
usertask.pop('Maximum scheduled energy per hour')


0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    1
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    1
47    1
48    1
49    1
Name: Maximum scheduled energy per hour, dtype: int64

In [4]:
usertask.dtypes

User & Task ID    object
Ready Time         int64
Deadline           int64
Energy Demand      int64
dtype: object

In [5]:
# construct lp problem for each user

for user_id in range(1,6):
    
    # lp_user1 = pulp.LpProblem("My_LP_Problem", pulp.LpMinimize)
    # ...
    # lp_user5 = pulp.LpProblem("My_LP_Problem", pulp.LpMinimize)
    exec("lp_user%s=pulp.LpProblem('My_LP_Problem', pulp.LpMinimize)"%user_id)


In [6]:
# construct decision variables 

task_by_hour = [[[]for j in range(24)] for i in range(5)]
task_by_id = [[[]for j in range(10)] for i in range(5)]

user_id = 1
task_id = 1

for task in usertask.itertuples():
    
    # construct decision variables for each task
    # if task1 of user1 starts from 20 to 23, then u1_t1_20, u1_t1_21, u1_t1_22 and u1_t1_23 will be crated
    for hour in range(task[2],task[3]+1):
        
        # u1_t1_20 = pulp.LpVariable('u1_t1_20', lowBound=0, upBound=1, cat='Continuous')
        # ...
        # u5_t10_23 = pulp.LpVariable('u5_t10_23', lowBound=0, upBound=1, cat='Continuous')
        exec("u%s_t%s_%s=pulp.LpVariable('u%s_t%s_%s', lowBound=0, upBound=1, cat='Continuous')"%(user_id,task_id,hour,user_id,task_id,hour))
        
        # store decision variables by hours
        exec("task_by_hour[%s][%s].append(u%s_t%s_%s)"%(user_id-1,hour,user_id,task_id,hour))
        
        # store decision variables by task id
        exec("task_by_id[%s][%s].append(u%s_t%s_%s)"%(user_id-1,task_id-1,user_id,task_id,hour))
        
    task_id += 1
    if task_id == 11:
        task_id = 1
        user_id += 1
        task


In [7]:
# construct constrain function

user_constrain = "0"
task_id = 1
user_id = 1

for task_energy in usertask['Energy Demand']:
    
    # construct constrain function for each task
    for task in task_by_id[user_id-1][task_id-1]:
        
        # add up all variables by current task id for current user
        exec("user_constrain += '+ %s'"%task)
    
    # construct constrain function (sum of variables of this task == energy demand of this task)
    exec("user_constrain += ' == %s'"%task_energy)
    
    # construct constrain function of current task for current user
    exec("lp_user%s += %s"%(user_id,user_constrain))
    
    task_id += 1
    if task_id == 11:
        task_id = 1
        user_id += 1
    user_constrain = "0"
    

In [8]:
# create folder

def mkdir(path): 
    import os
 
    path=path.strip()
    path=path.rstrip("\\")

    if not os.path.exists(path):
        os.makedirs(path)  
        return True

In [9]:
# plot scheduling results

def plot_schedule(energy_usage,plt_name):
    
    energy_usage.append(0)
    
    # set x-coordinate scale label
    hour_list = list(map(int, range(25)))
    
    # set dpi and size of figure
    plt.figure(dpi = 300, figsize = (15,10))

    # set y-coordinate to int
    plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
    
    # set y-coordinate interval to 1
    plt.gca().yaxis.set_major_locator(MultipleLocator(1))

    # set coordinate label
    plt.xlabel('Time (H)')
    plt.ylabel('Total Power (KW)')
    
    # draw bar chart
    plt.bar(range(25), energy_usage, width = 1,align = 'edge',linewidth = 1,edgecolor = 'black' , tick_label = hour_list)

    
    # save the bar chart
    mkdir('./charts_for_abnormal_prices/')
    plt.savefig('./charts_for_abnormal_prices/energy_usage(guideline%s).jpg' % plt_name)   
    plt.close()



In [10]:
# construct objective function and do calculation

tasklist = "0"
user_function = "0"
energy_usage = [0]*24

for price in prices.itertuples():
    
    for user_id in range(1,6):
        
        for hour in range(0,24):
            
            for task in task_by_hour[user_id-1][hour]:
                
                # add up variables in current hour for current user 
                exec("tasklist += '+ %s'"%task)
            
            # add up (price * sum of variables) of each hour for current user, to construct objective function
            exec("user_function += '+ (%s) * %s'"%(tasklist,price[hour+1]))
            
            # reset the sum of variables of current hour
            tasklist = "0"
       
        # construct objective function for current user using current price guideline 
        exec("lp_user%s += %s"%(user_id,user_function))
        
        # reset objective function
        user_function = "0"
        
        # calculate scheduling results for current user using current price guideline 
        exec("lp_user%s.solve()"%user_id)
        
        # add up scheduling results of current user by hours
        for hour in range(0,24):
            for task in task_by_hour[user_id-1][hour]:
                energy_usage[hour] += task.varValue
    
    # plot sum of scheduling results of all 5 users by hours using current price guideline
    plot_schedule(energy_usage,price[0])
    energy_usage = [0]*24




Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/reol/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/zy/j2jkps316bzfk9v5fvqrr8rm0000gn/T/1bbbed9908aa4199954380938bc7fb85-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/zy/j2jkps316bzfk9v5fvqrr8rm0000gn/T/1bbbed9908aa4199954380938bc7fb85-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 138 RHS
At line 149 BOUNDS
At line 211 ENDATA
Problem MODEL has 10 rows, 61 columns and 61 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 8 (-2) rows, 54 (-7) columns and 54 (-7) elements
0  Obj 10.247654 Primal inf 19.999992 (8)
8  Obj 100.87087
Optimal - objective value 100.87087
After Postsolve, objective 100.87087, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 100.8708686 - 8 iterations time 0.002, Pr

/Users/reol/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/reol/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/zy/j2jkps316bzfk9v5fvqrr8rm0000gn/T/7604be6510e24f678b4bea239c51fac0-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/zy/j2jkps316bzfk9v5fvqrr8rm0000gn/T/7604be6510e24f678b4bea239c51fac0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 138 RHS
At line 149 BOUNDS
At line 211 ENDATA
Problem MODEL has 10 rows, 61 columns and 61 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 8 (-2) rows, 54 (-7) columns and 54 (-7) elements
0  Obj 10.296194 Primal inf 19.999992 (8)
8  Obj 100.97748
Optimal - objective value 100.97748
After Postsolve, objective 100.97748, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 100.9774757 - 8 iterations time 0.002, Pr